<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Подключаемые все необходимые для выполнения исследования библиотеки:

In [26]:
import pandas as pd # работа с табличными данными
import numpy as np # работа с векторами и матрицами 
from sklearn.model_selection import train_test_split # выделение обучающей и тестовой выборок
from sklearn.model_selection import cross_val_score # кросс-валидация
from sklearn.linear_model import LinearRegression # модель линейной регрессии
from sklearn.metrics import r2_score # коэффициент детерминации 
from sklearn.preprocessing import StandardScaler # масштабирование признаков

Загружаем данные из файла и выводим общую информацию:

In [27]:
data = pd.read_csv('/datasets/insurance.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Изучаем данные:

In [28]:
data.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Определяем, в каких признаках есть пропуски:

In [29]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Проверяем на наличие явных дубликатов:

In [30]:
data.duplicated().sum()

153

Избавляемся от явных дубликатов:

In [31]:
data = data.drop_duplicates()

Проверяем, что явных дубликатов не осталось:

In [32]:
data.duplicated().sum()

0

Переименуем столбцы:

In [33]:
data = data.rename(columns=
                   {'Пол': 'sex', 
                    'Возраст': 'age', 
                    'Зарплата': 'salary', 
                    'Члены семьи': 'family_members',
                    'Страховые выплаты': 'insurance_payments'
                   }
                  )

**Выводы**

Исходный набор данных содержит 5 000 наблюдений по 5-ти признакам. Категориальный признак пол представлен числом, подготовительный этап по кодированию не нужен. Пропусков нет, но есть дубликаты, хотя из-за отсутствия уникального идетификатора клиента нельзя утвержать со 100% уверенностью, что они явные. 

## Умножение матриц

Докажем аналитически, что при умножении признаков на случайную обратимую матрицу качество линейной регрессии не изменится.

Нужно доказать, что прогнозы модели на исходных признаках $a$ тождественно равны прогнозам модели на преобразованных признаках $a'$:

$$
a = a'
$$

$$
a = Xw
$$

$$
a' = X'w' = XPw'
$$

Где $X$ - матрица исходных признаков, $X'$ - матрица преобразованных признаков; $P$ - случайная обратимая матрица; $w$ - вектор весов (коэффициентов) исходной модели; $w'$ - вектор весов модели на преобразованных признаках.

Из аналитического решения линейной регрессии:

$$
w = (X^T X)^{-1} X^T y
$$

$$
w' = ((XP)^T (XP))^{-1} (XP)^T y
$$

Раскроем скобки для $w'$. Транспонированная матрица произведения равна произведению транспонированных матриц сомножителей, взятых в обратном порядке $(AB)^T=B^T A^T$:

$$
w' = ((XP)^T (XP))^{-1} (XP)^T y
$$

$$
w' = (P^T X^T XP)^{-1} P^T X^T y
$$

Используем ассоциативное (сочетательное) свойство:

$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$

Обратное значение произведения обратимых матриц A и B равно значению, полученному при умножении матриц, обратных исходным $(AB)^{-1} = B^{-1} A^{-1}$:

Обратная матрица транспонированной матрицы эквивалентна транспонированной обратной матрице $(P^T)^{-1} = (P^{-1})^T$:

$$
w' = P^{-1}(X^T X)^{-1} (P^{-1})^T P^T X^T y
$$

Используем сочетательное свойство - сначала умножаем $(P^{-1})^T P^T$, чтобы воспользоваться свойством $(P^{-1})^T P^T = (PP^{-1})^{T} = E^T = E$:

$$
w' = P^{-1}(X^T X)^{-1} ((P^{-1})^T P^T) X^T y
$$

$$
w' = P^{-1}(X^T X)^{-1}(PP^{-1})^T X^T y
$$

$$
w' = P^{-1}(X^T X)^{-1} E^T X^T y
$$

$$
w' = P^{-1}(X^T X)^{-1} E X^T y
$$

Используем сочетательное свойство - сначала умножаем $E X^T$, чтобы воспользоваться свойством $E X^T = X^T$:

$$
w' = P^{-1}(X^T X)^{-1} (E X^T) y
$$

$$
w' = P^{-1}(X^T X)^{-1} X^T y
$$

Заметим, что в полученном выражении $(X^T X)^{-1} X^T y$ - это $w$ (из аналитического решения линейной регрессии в случае w), поэтому $w'$ можно выразить через $w$:

$$
w' = P^{-1}(X^T X)^{-1} X^T y
$$

$$
w = (X^T X)^{-1} X^T y
$$

$$
w' = P^{-1}((X^T X)^{-1} X^T y) = P^{-1} w
$$

Подставим полученное значение в формулу для $a'$:

$$
a' = XPw'
$$

$$
w' = P^{-1} w
$$

$$
a' = X P P^{-1} w
$$

$$
a' = X (P P^{-1}) w
$$

$$
a' = X E w
$$

$$
a' = X w
$$

Полученное выражение равно формуле для $a$:

$$
a = X w
$$

$$
a' = X w
$$

$$
a = a'
$$

Что и требовалось доказать.

**Выводы**

Из аналитического решения следует, что качество линейной регресии после преобразования признаков умножением на обратимую матрицу не должно изменится. На следующем этапе проверим это на практике.

## Алгоритм преобразования

Для преобразования дотаточно будет умножить матрицу исходных признаков на случайную обратимую матрицу:

$$
X' = XP
$$

Тогда исходные значения признаков можно восстановить умножением на обратную матрицу:

$$
X = X' P^{-1} = XP P^{-1} = X(P P^{-1}) = X E = X
$$

## Проверка алгоритма

Выделяем регрессоры и целевой признак, преобразуем в матрицу и в вектор:

In [34]:
features = data.drop('insurance_payments', axis=1)
target = data['insurance_payments']

X = features.values
y = target.values

Напишем функцию для получения квадратной обратимой матрицы:

In [35]:
def get_invertible_matrix(size):
    np.random.seed(12345)
    det = 0
    # проверяем, чтобы определитель не был равен 0 - признак обратимости
    while det == 0:
        matrix = np.random.rand(size, size)
        # находим определитель матрицы
        det = np.linalg.det(matrix)
        return matrix

Вычисляем  квадратную матрицу:

In [36]:
square_matrix = get_invertible_matrix(X.shape[1])

Преобразуем признаки - умножаем их на обратимую матрицу:

In [37]:
X_new = X.dot(square_matrix)

Выделяем обучающий и тестовый набор данных для исходных и преобразованных признаков:

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=12345)
X_train_new, X_test_new, y_train, y_test = train_test_split(X_new, y, test_size=0.33, random_state=12345)

Стандартизируем численные признаки:

In [39]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

scaler.fit(X_train_new)
X_train_new = scaler.transform(X_train_new)
X_test_new = scaler.transform(X_test_new)

Доказываем, что качество моделей не изменится на практике:

Качество модели до преобразования признаков:

In [40]:
model_lr = LinearRegression() # инициализация модели линейной регрессии
scores = cross_val_score(model_lr, X, y, cv=5, scoring='r2') 
final_score = scores.mean()
print('Значение метрики R2 для модели линейной регрессии до преобразования на обучающей выборке:', final_score)

Значение метрики R2 для модели линейной регрессии до преобразования на обучающей выборке: 0.4277942580280493


In [41]:
model_lr = LinearRegression() # инициализация модели линейной регрессии
model_lr.fit(X_train, y_train)
predictions_test = model_lr.predict(X_test)

score = r2_score(y_test, predictions_test)

print('R2 модели до преобразования признаков на тествой выборке', score)

R2 модели до преобразования признаков на тествой выборке 0.43460146736926875


Качество модели после преобразования признаков (умножения на обратимую матрицу):

In [42]:
model_lr = LinearRegression() # инициализация модели линейной регрессии
scores = cross_val_score(model_lr, X_train_new, y_train, cv=5, scoring='r2') 
final_score = scores.mean()
print('Значение метрики R2 для модели линейной регрессии после преобразования на обучающей выборке:', final_score)

Значение метрики R2 для модели линейной регрессии после преобразования на обучающей выборке: 0.42501725798126105


In [43]:
model_lr = LinearRegression() # инициализация модели линейной регрессии
model_lr.fit(X_train_new, y_train)
predictions_test = model_lr.predict(X_test_new)

score = r2_score(y_test, predictions_test)

print('R2 модели после преобразования признаков на тествой выборке', score)

R2 модели после преобразования признаков на тествой выборке 0.4346014673689439


**Выводы**

Качество линейной регресии после преобразования признаков умножением на обратимую матрицу практически не изменилось.

Проверим, что после умножения на обратную матрицу восстанавливаем исходные значения:

In [44]:
np.isclose(X, X_new.dot(np.linalg.inv(square_matrix)))

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       ...,
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

**Выводы**

После уможения на обратную матрицу удаётся восстановить исходные значения признаков.